In [224]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [225]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [226]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [227]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [228]:
for dataset in [train_data,test_data]:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms', 'Mme'], 'Miss')
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

title_mapping={"Mr":1, "Miss":2, "Mrs":3, "Master":4, "Rare":5}
for dataset in [train_data,test_data]:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Nandhu\AppData\Local\Temp\ipykernel_18684\2579466297.py:2: SyntaxWarning: invalid escape sequence '\.'
  dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [229]:
#combine Parch & SibSp- to get sense of passenger family size
for dataset in [train_data,test_data]:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

In [230]:
# Feature Engineering
# lets keep OG df safe
train_impute = train_data.copy()
test_impute = test_data.copy()

# Combine them for imputation process
combined_data = pd.concat([train_impute.drop('Survived', axis=1), test_impute], ignore_index=True)

# --- 1. Isolate the data ---
# Separate the data into two sets: one for training our imputer, one for predicting
age_known_df = combined_data[combined_data['Age'].notna()]
age_missing_df = combined_data[combined_data['Age'].isna()]


In [231]:

print(f"Number of passengers with known age: {len(age_known_df)}")
print(f"Number of passengers with missing age: {len(age_missing_df)}")

Number of passengers with known age: 1046
Number of passengers with missing age: 263


In [232]:
from sklearn.ensemble import RandomForestRegressor
# 2. Select features for imputation
# These features will be used to PREDICT age
imputation_features = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Title', 'FamilySize']
combined_data['Fare'].fillna(combined_data['Fare'].median(), inplace=True)

# Define features (X_age) and target(y_age) for imputer model
X_age_train = age_known_df[imputation_features]
y_age_train = age_known_df['Age']
X_age_predict = age_missing_df[imputation_features]

# 3.Train Regression model
age_regressor = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
age_regressor.fit(X_age_train, y_age_train)


C:\Users\Nandhu\AppData\Local\Temp\ipykernel_18684\1642706708.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['Fare'].fillna(combined_data['Fare'].median(), inplace=True)


RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)

In [233]:
# 4. Predict Missing Ages

predicted_ages = age_regressor.predict(X_age_predict)

In [234]:
# 5. Fill Missing Values
combined_data.loc[combined_data['Age'].isna(), 'Age'] = predicted_ages

In [235]:
# Verification

print(f"\nMissing 'Age' values remaining in combined data: {combined_data['Age'].isnull().sum()}")



Missing 'Age' values remaining in combined data: 0


In [236]:
# 6. Update your OG train_data and test_data

train_data['Age'] = combined_data.iloc[:len(train_data)]['Age']
test_data['Age'] = combined_data.iloc[len(train_data):]['Age']

In [237]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,1.0,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,3.0,2,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,2.0,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,3.0,2,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,1.0,1,1


In [238]:
# cabin- first letter represents the class A,B,C. missing vals are denoted by U for now

for dataset in [train_data,test_data]:
    dataset['Deck'] = dataset['Cabin'].str[0]
    dataset['Deck'] = dataset['Deck'].fillna('U') # Fill NaN with 'U'
deck_mapping = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "T": 8, "U": 0}
for dataset in [train_data, test_data]:
    dataset['Deck'] = dataset['Deck'].map(deck_mapping)

In [239]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          0 non-null      float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  Title        418 non-null    int64  
 12  FamilySize   418 non-null    int64  
 13  IsAlone      418 non-null    int64  
 14  Deck         418 non-null    int64  
dtypes: float64(2), int64(9), object(4)
memory usage: 49.1+ KB


In [240]:
test_data.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Title,FamilySize,IsAlone,Deck
count,418.000000,418.000000,418.000000,0.0,418.000000,418.000000,417.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,0.363636,NaN,0.447368,0.392344,35.627188,1.741627,1.839713,0.605263,0.717703
std,120.810458,0.841838,0.481622,NaN,0.896760,0.981429,55.907576,1.006091,1.519072,0.489380,1.508321
min,892.000000,1.000000,0.000000,NaN,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,996.250000,1.000000,0.000000,NaN,0.000000,0.000000,7.895800,1.000000,1.000000,0.000000,0.000000
50%,1100.500000,3.000000,0.000000,NaN,0.000000,0.000000,14.454200,1.000000,1.000000,1.000000,0.000000
75%,1204.750000,3.000000,1.000000,NaN,1.000000,0.000000,31.500000,2.000000,2.000000,1.000000,0.000000
max,1309.000000,3.000000,1.000000,NaN,8.000000,9.000000,512.329200,5.000000,11.000000,1.000000,7.000000


In [241]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [242]:
# for adv age imputer model

y = train_data['Survived']
X = train_data.drop(columns=['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

categorical_features = ['Embarked']
numeric_features = ['Age', 'Fare'] # ONLY the true continuous features
passthrough_features = ['Pclass', 'Title', 'FamilySize', 'IsAlone', 'Sex','Deck'] # The rest

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())]) # No imputer needed now!

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('pass', 'passthrough', passthrough_features)
    ],
    remainder='drop'
)

In [243]:

# # median imputer 

# features_to_drop = ['Survived', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']
# X = train_data.drop(columns=features_to_drop, errors='ignore',axis=1)
# y = train_data['Survived']
# categorical_features = ['Embarked'] # We'll bring these back just for the pipeline
# numeric_features = ['Age', 'Fare', 'Pclass', 'Title', 'FamilySize', 'IsAlone', 'Sex']


# # Create preprocessing pipelines
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Create the master preprocessor
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ],
#     remainder='drop'
# )

In [244]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [245]:
model_performance={}

In [246]:
from sklearn.linear_model import LogisticRegression
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

# Train the model
lr_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('pass', 'passthrough',
                                                  ['Pclass', 'Title',
                                                   'FamilySize', 'IsAlone',
                                                   'Sex', 'Deck'])])),
                ('classifier', LogisticRegression(random_state=42))])

In [247]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_lr = lr_pipeline.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
model_performance['Logistic Regression'] = accuracy_lr
print("Accuracy of Logistic Regression:", accuracy_lr)
print(classification_report(y_test, y_pred_lr))

Accuracy of Logistic Regression: 0.7988826815642458
              precision    recall  f1-score   support

           0       0.83      0.82      0.83       105
           1       0.75      0.77      0.76        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [248]:
"""
    Random Forest Classifier()
    
"""
from sklearn.ensemble import RandomForestClassifier
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train the model
rf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('pass', 'passthrough',
                                                  ['Pclass', 'Title',
                                                   'FamilySize', 'IsAlone',
                                                   'Sex', 'Deck'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [249]:
y_pred_rfc = rf_pipeline.predict(X_test)
accuracy_rfc = accuracy_score(y_test, y_pred_rfc)
model_performance['Random Forest Classifier'] = accuracy_rfc
print("Accuracy of Random Forest Classifier:", accuracy_rfc)
print(classification_report(y_test, y_pred_rfc))

Accuracy of Random Forest Classifier: 0.8268156424581006
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       105
           1       0.78      0.81      0.79        74

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179



In [250]:
from sklearn.model_selection import GridSearchCV

param_grid_rf = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__min_samples_split': [2, 5, 10]
}
pipeline_for_grid = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
grid_search_rf = GridSearchCV(pipeline_for_grid, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked']),
                                                                        ('pass',
                                                                         'passthrough',
                                                                         ['Pclass',
                                                                          'Title',
                                                                          'FamilySize',
                                                                          'IsAlone',
                                                                          'Sex',
                                                                          'Deck'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [5, 10, 15],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [251]:
print(f"Best Parameters found: {grid_search_rf.best_params_}")
print(f"Best CV Accuracy: {grid_search_rf.best_score_:.4f}")

Best Parameters found: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
Best CV Accuracy: 0.8357


In [252]:
print("Accuracy:", grid_search_rf.best_estimator_.score(X_test, y_test))

Accuracy: 0.8379888268156425


In [253]:
y_pred = grid_search_rf.best_estimator_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
model_performance['RFC Grid Search'] = accuracy_score(y_test, y_pred)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8379888268156425
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       105
           1       0.83      0.77      0.80        74

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [254]:
# adv hyperparameter tuning for Random Forest

param_grid_adv_rf={
    'classifier__n_estimators': [250, 300, 350],
    'classifier__max_depth':  [4, 5, 6, 7],
    'classifier__min_samples_leaf': [1, 2, 3],
    'classifier__min_samples_split':  [8, 10, 12],
    'classifier__max_features': ['sqrt', 'log2', None]
}

In [255]:

pipeline_for_adv_grid = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
grid_search_adv_rf = GridSearchCV(pipeline_for_adv_grid, param_grid_adv_rf, cv=5, scoring='accuracy', n_jobs=-1,verbose=1)
grid_search_adv_rf.fit(X_train, y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked']),
                                                                        ('pass',
                                                                         'passthrough',
                                                                         ['Pclass',
                                                                          'Title',
                                                                          'FamilySize',
                                                                          'IsAlone',
                                                                          'Sex',
                                                                          'Deck'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [4, 5, 6, 7],
                         'classifier__max_features': ['sqrt', 'log2', None],
                         'classifier__min_samples_leaf': [1, 2, 3],
                         'classifier__min_samples_split': [8, 10, 12],
                         'classifier__n_estimators': [250, 300, 350]},
             scoring='accuracy', verbose=1)

In [256]:
print(f"Best Parameters found: {grid_search_adv_rf.best_params_}")
print(f"Best CV Accuracy: {grid_search_adv_rf.best_score_:.4f}")

Best Parameters found: {'classifier__max_depth': 7, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 3, 'classifier__min_samples_split': 12, 'classifier__n_estimators': 250}
Best CV Accuracy: 0.8384


In [257]:
y_pred_adv_rf = grid_search_adv_rf.best_estimator_.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_adv_rf)
print("Accuracy:", accuracy)
model_performance['RFC Advanced Grid Search'] = accuracy
print("Classification Report:")
print(classification_report(y_test, y_pred_adv_rf))

Accuracy: 0.8379888268156425
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       105
           1       0.83      0.77      0.80        74

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [258]:
"""
    SVM Classifier
"""
from sklearn.svm import SVC

svc_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42))
])

# Train the model
svc_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('pass', 'passthrough',
                                                  ['Pclass', 'Title',
                                                   'FamilySize', 'IsAlone',
                                                   'Sex', 'Deck'])])),
                ('classifier', SVC(random_state=42))])

In [259]:
y_pred_svc = svc_pipeline.predict(X_test)

In [260]:
accuracy = accuracy_score(y_test, y_pred_svc)
model_performance['SVM (Default)'] = accuracy
print("\n--- SVM (Default) Performance ---")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred_svc))


--- SVM (Default) Performance ---
Accuracy: 0.8212
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       105
           1       0.80      0.76      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



In [261]:
param_grid_svc = {
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__gamma': ['scale', 'auto', 0.1, 1],
    'classifier__kernel': ['rbf', 'poly']
}

# Create a pipeline with a placeholder
pipeline_for_grid_svc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42, probability=True)) # Added probability=True for later use in soft voting
])

grid_search_svc = GridSearchCV(pipeline_for_grid_svc, param_grid_svc, cv=5, scoring='accuracy')
grid_search_svc.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked']),
                                                                        ('pass',
                                                                         'passthrough',
                                                                         ['Pclass',
                                                                          'Title',
                                                                          'FamilySize',
                                                                          'IsAlone',
                                                                          'Sex',
                                                                          'Deck'])])),
                                       ('classifier',
                                        SVC(probability=True,
                                            random_state=42))]),
             param_grid={'classifier__C': [0.1, 1, 10, 100],
                         'classifier__gamma': ['scale', 'auto', 0.1, 1],
                         'classifier__kernel': ['rbf', 'poly']},
             scoring='accuracy')

In [262]:
print("Best Parameters:", grid_search_svc.best_params_)
print("Best Accuracy:", grid_search_svc.best_score_)

Best Parameters: {'classifier__C': 0.1, 'classifier__gamma': 'auto', 'classifier__kernel': 'poly'}
Best Accuracy: 0.8314192849404117


In [263]:

y_pred_svc_grid = grid_search_svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_svc_grid)
model_performance['SVM (GridSearch)'] = accuracy
print("\n--- SVM (GridSearch) Test Set Performance ---")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred_svc_grid))


--- SVM (GridSearch) Test Set Performance ---
Accuracy: 0.8101
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



In [264]:
# advanced hyperparameter tuning for svm
param_grid_adv_svc=[
    # Grid 1: Focused search for the 'rbf' kernel
    {
        'classifier__kernel': ['rbf'],
        'classifier__C': [5, 10, 15, 20],  # Values around our previous best of 10
        'classifier__gamma': [0.01, 0.05, 0.1, 'scale', 'auto'] # Finer-grained gamma
    },
    # Grid 2: Focused search for the 'poly' kernel with different degrees
    {
        'classifier__kernel': ['poly'],
        'classifier__C': [1, 5, 10],
        'classifier__degree': [2, 3, 4], # The 'poly' kernel's most important parameter
        'classifier__gamma': ['scale', 'auto']
    }
]

In [265]:
pipeline_for_grid_adv_svc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42, probability=True))
])

grid_search_adv_svc = GridSearchCV(pipeline_for_grid_adv_svc, param_grid_adv_svc, cv=5, scoring='accuracy',n_jobs=1,verbose=1)
grid_search_adv_svc.fit(X_train, y_train)

Fitting 5 folds for each of 38 candidates, totalling 190 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked']),
                                                                        ('pass',
                                                                         'passthrough',
                                                                         ['Pclass',
                                                                          'Title',
                                                                          'FamilyS...
                                                                          'Deck'])])),
                                       ('classifier',
                                        SVC(probability=True,
                                            random_state=42))]),
             n_jobs=1,
             param_grid=[{'classifier__C': [5, 10, 15, 20],
                          'classifier__gamma': [0.01, 0.05, 0.1, 'scale',
                                                'auto'],
                          'classifier__kernel': ['rbf']},
                         {'classifier__C': [1, 5, 10],
                          'classifier__degree': [2, 3, 4],
                          'classifier__gamma': ['scale', 'auto'],
                          'classifier__kernel': ['poly']}],
             scoring='accuracy', verbose=1)

In [266]:
print("Best Parameters:", grid_search_adv_svc.best_params_)
print("Best Accuracy:", grid_search_adv_svc.best_score_)

Best Parameters: {'classifier__C': 15, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf'}
Best Accuracy: 0.8342263370432385


In [267]:

y_pred_svc_adv_grid = grid_search_adv_svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_svc_adv_grid)
model_performance['SVM Adv (GridSearch)'] = accuracy
print("\n--- SVM Adv(GridSearch) Test Set Performance ---")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred_svc_adv_grid))
""".
    seems like the model is overfitting
    based on ananlysis my best model is SVM (GridSearch) which gave 0.8268 accuracy
"""


--- SVM Adv(GridSearch) Test Set Performance ---
Accuracy: 0.8156
              precision    recall  f1-score   support

           0       0.83      0.86      0.85       105
           1       0.79      0.76      0.77        74

    accuracy                           0.82       179
   macro avg       0.81      0.81      0.81       179
weighted avg       0.81      0.82      0.82       179



'.\n    seems like the model is overfitting\n    based on ananlysis my best model is SVM (GridSearch) which gave 0.8268 accuracy\n'

In [268]:
""".
    Decision Tree Classifier
"""
from sklearn.tree import DecisionTreeClassifier
dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Train the model
dt_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('pass', 'passthrough',
                                                  ['Pclass', 'Title',
                                                   'FamilySize', 'IsAlone',
                                                   'Sex', 'Deck'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [269]:

y_pred_dt = dt_pipeline.predict(X_test)

In [270]:
accuracy = accuracy_score(y_test, y_pred_dt)
model_performance['Decision Tree'] = accuracy
print("\n--- Decision Tree Performance ---")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred_dt))


--- Decision Tree Performance ---
Accuracy: 0.7765
              precision    recall  f1-score   support

           0       0.82      0.79      0.81       105
           1       0.72      0.76      0.74        74

    accuracy                           0.78       179
   macro avg       0.77      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179



In [271]:
#XGBoost Classifier
from xgboost import XGBClassifier
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

# Train the model
xgb_pipeline.fit(X_train, y_train)

c:\Users\Nandhu\anaconda3\envs\mlenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:43:10] WARNING: D:\bld\xgboost-split_1745056718345\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('pass', 'passthrough',
                                                  ['Pclass', 'Title',
                                                   'FamilySize', 'IsAlone',
                                                   'Sex', 'Deck']...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [272]:

xgb_scores = cross_val_score(xgb_pipeline, X, y, cv=5, scoring='accuracy')
xgb_accuracy = xgb_scores.mean()
model_performance['XGBoost'] = xgb_accuracy
print("\n--- XGBoost Performance ---")
print(f"Accuracy: {xgb_accuracy:.4f}")

c:\Users\Nandhu\anaconda3\envs\mlenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:43:11] WARNING: D:\bld\xgboost-split_1745056718345\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Nandhu\anaconda3\envs\mlenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:43:11] WARNING: D:\bld\xgboost-split_1745056718345\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Nandhu\anaconda3\envs\mlenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:43:11] WARNING: D:\bld\xgboost-split_1745056718345\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Nandhu\anaconda3\envs\mlenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:43:11] WARNING: D:\bld\xgboost-split_1745056718345\work\src\learner.cc:738: 
Parameters: { "use_


--- XGBoost Performance ---
Accuracy: 0.8148


c:\Users\Nandhu\anaconda3\envs\mlenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:43:11] WARNING: D:\bld\xgboost-split_1745056718345\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [273]:
#hyperparameter tuning for XGBoost

param_grid_xgb = {
    'classifier__learning_rate': [0.05, 0.1, 0.2],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5, 7],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0]
}
xgb_pipeline_grid = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42, eval_metric='logloss'))
])
grid_search_xgb = GridSearchCV(xgb_pipeline_grid, param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked']),
                                                                        ('pass',
                                                                         'passthrough',
                                                                         ['Pclass',
                                                                          'Title',
                                                                          'FamilyS...
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None, ...))]),
             n_jobs=-1,
             param_grid={'classifier__colsample_bytree': [0.8, 1.0],
                         'classifier__learning_rate': [0.05, 0.1, 0.2],
                         'classifier__max_depth': [3, 5, 7],
                         'classifier__n_estimators': [100, 200, 300],
                         'classifier__subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [274]:

print(f"Best CV Accuracy from XGBoost search: {grid_search_xgb.best_score_:.4f}")
print(f"Best Parameters found: {grid_search_xgb.best_params_}")

Best CV Accuracy from XGBoost search: 0.8371
Best Parameters found: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.05, 'classifier__max_depth': 3, 'classifier__n_estimators': 200, 'classifier__subsample': 0.8}


In [275]:
# ensemble methods
from sklearn.ensemble import VotingClassifier

best_rf_from_grid = grid_search_rf.best_estimator_
best_svc_hyperparams = grid_search_svc.best_params_
svc_params_for_new_model = {
    key.replace('classifier__', ''): value 
    for key, value in best_svc_hyperparams.items()
}
svc_for_voting = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(**svc_params_for_new_model, probability=True, random_state=42))
])
estimators = [
    ('lr', lr_pipeline),
    ('rf', best_rf_from_grid),
    ('svc', svc_for_voting),
    ('dt', dt_pipeline)
]
voting_clf = VotingClassifier(estimators=estimators, voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('num',
                                                                                Pipeline(steps=[('scaler',
                                                                                                 StandardScaler())]),
                                                                                ['Age',
                                                                                 'Fare']),
                                                                               ('cat',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='most_frequent')),
                                                                                                ('onehot',
                                                                                                 OneHotEncoder(handle_unknown='ignore'))]),
                                                                                ['Embarked']),
                                                                               ('pass',
                                                                                'passthrough',
                                                                                ['Pclass',
                                                                                 'Title',
                                                                                 '...
                                                                                Pipeline(steps=[('scaler',
                                                                                                 StandardScaler())]),
                                                                                ['Age',
                                                                                 'Fare']),
                                                                               ('cat',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='most_frequent')),
                                                                                                ('onehot',
                                                                                                 OneHotEncoder(handle_unknown='ignore'))]),
                                                                                ['Embarked']),
                                                                               ('pass',
                                                                                'passthrough',
                                                                                ['Pclass',
                                                                                 'Title',
                                                                                 'FamilySize',
                                                                                 'IsAlone',
                                                                                 'Sex',
                                                                                 'Deck'])])),
                                              ('classifier',
                                               DecisionTreeClassifier(random_state=42))]))],
                 voting='soft')

In [276]:
y_pred_voting = voting_clf.predict(X_test)

In [277]:
accuracy = accuracy_score(y_test, y_pred_voting)
model_performance['Voting Classifier'] = accuracy
print("\n--- Voting Classifier Ensemble Performance ---")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred_voting))


--- Voting Classifier Ensemble Performance ---
Accuracy: 0.8324
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       105
           1       0.81      0.78      0.79        74

    accuracy                           0.83       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.83      0.83      0.83       179



In [278]:
# modeify ensemble with our best tuned models
# best rfc
champion_rfc=grid_search_adv_rf.best_estimator_
# best svc
champion_svc=grid_search_adv_svc.best_estimator_
svc_params_for_new_model = {
    key.replace('classifier__', ''): value 
    for key, value in best_svc_hyperparams.items()
}
champion_svc_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(**svc_params_for_new_model, probability=True, random_state=42))
])
# best xgb
champion_xgb=grid_search_xgb.best_estimator_

estimators = [
    ('rfc', champion_rfc),
    ('svc', champion_svc),
    ('xgb', champion_xgb)
]
champion_ensemble = VotingClassifier(estimators=estimators, voting='soft',weights=[0.35, 0.35, 0.30], n_jobs=-1)
champion_ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('rfc',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('num',
                                                                                Pipeline(steps=[('scaler',
                                                                                                 StandardScaler())]),
                                                                                ['Age',
                                                                                 'Fare']),
                                                                               ('cat',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='most_frequent')),
                                                                                                ('onehot',
                                                                                                 OneHotEncoder(handle_unknown='ignore'))]),
                                                                                ['Embarked']),
                                                                               ('pass',
                                                                                'passthrough',
                                                                                ['Pclass',
                                                                                 'Title',...
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.05,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=3,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=200,
                                                             n_jobs=None,
                                                             num_parallel_tree=None, ...))]))],
                 n_jobs=-1, voting='soft', weights=[0.35, 0.35, 0.3])

In [279]:
y_pred_adv_voting=champion_ensemble.predict(X_test)

In [280]:
accuracy = accuracy_score(y_test, y_pred_adv_voting)
model_performance['Adv Voting Classifier'] = accuracy
print("\n--- Adv Voting Classifier Ensemble Performance ---")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred_adv_voting))


--- Adv Voting Classifier Ensemble Performance ---
Accuracy: 0.8268
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       105
           1       0.81      0.76      0.78        74

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179



In [281]:
performance_df = pd.DataFrame.from_dict(model_performance, orient='index', columns=['Accuracy'])
print(performance_df.sort_values(by='Accuracy', ascending=False))

                          Accuracy
RFC Grid Search           0.837989
RFC Advanced Grid Search  0.837989
Voting Classifier         0.832402
Adv Voting Classifier     0.826816
Random Forest Classifier  0.826816
SVM (Default)             0.821229
SVM Adv (GridSearch)      0.815642
XGBoost                   0.814814
SVM (GridSearch)          0.810056
Logistic Regression       0.798883
Decision Tree             0.776536


In [282]:
print(breakdown)

NameError: name 'breakdown' is not defined

In [283]:
print("\nRetraining the best model on the full dataset...")
# best_model_pipeline = grid_search_svc.best_estimator_ # or whichever model performed best
best_model_pipeline= grid_search_rf.best_estimator_
best_model_pipeline.fit(X, y)


Retraining the best model on the full dataset...


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked']),
                                                 ('pass', 'passthrough',
                                                  ['Pclass', 'Title',
                                                   'FamilySize', 'IsAlone',
                                                   'Sex', 'Deck'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=10, min_samples_leaf=2,
                                        random_state=42))])

In [284]:
final_test_X = test_data[X.columns]

In [285]:
final_predictions = best_model_pipeline.predict(final_test_X)

In [286]:
submission_df = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": final_predictions
})

In [287]:
submission_df.to_csv("submission.csv", index=False)